In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from scipy.special import j0  # 第1種0次ベッセル関数
from scipy.optimize import curve_fit
import pandas as pd
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# 日本語フォントの設定
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False

print("ライブラリのインポートが完了しました！")

## 1. 微動観測の基礎

### 1.1 微動とは？
微動は地盤に常時存在する微小な振動です。主な発生源は：
- 海の波（0.1-1 Hz）
- 交通振動（1-20 Hz）
- 風、人間活動など

### 1.2 なぜ微動を使うのか？
- 人工的な振源が不要
- 低周波数成分が豊富（深部構造の探査に有利）
- 都市部でも観測可能

まず、模擬的な微動データを生成してみましょう。

In [ ]:
# 模擬微動データの生成
def generate_synthetic_microtremor(duration=300, fs=100, frequencies=[0.5, 1.0, 2.0, 5.0], 
                                  amplitudes=[1.0, 0.5, 0.3, 0.1]):
    """
    合成微動データを生成する関数
    
    Parameters:
    - duration: 記録時間 (秒)
    - fs: サンプリング周波数 (Hz)
    - frequencies: 含まれる周波数成分 (Hz)
    - amplitudes: 各周波数成分の振幅
    """
    t = np.arange(0, duration, 1/fs)
    data = np.zeros(len(t))
    
    # 複数の正弦波を重ね合わせて微動を模擬
    for f, a in zip(frequencies, amplitudes):
        phase = np.random.rand() * 2 * np.pi
        data += a * np.sin(2 * np.pi * f * t + phase)
    
    # ノイズを追加
    data += 0.1 * np.random.randn(len(t))
    
    return t, data

# データ生成と可視化
t, microtremor = generate_synthetic_microtremor()

# 時系列データの表示
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(t[:1000], microtremor[:1000])
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.title('Microtremor Time Series (First 10 seconds)')
plt.grid(True)

# スペクトルの計算と表示
plt.subplot(1, 2, 2)
f, psd = signal.welch(microtremor, fs=100, nperseg=1024)
plt.semilogy(f, psd)
plt.xlabel('Frequency (Hz)')
plt.ylabel('PSD')
plt.title('Power Spectral Density')
plt.grid(True)
plt.xlim(0, 10)

plt.tight_layout()
plt.show()

## 2. アレイ観測の概念

微動アレイ観測では、複数の地震計を配置して同時観測を行います。
一般的な配置：
- 円形アレイ（中心＋円周上）
- 三角形アレイ
- L字型アレイ

ここでは、円形アレイを例に解析を進めます。

# 微動アレイ観測解析チュートリアル

このノートブックでは、微動アレイ観測データの解析手法を実践的に学びます。

## 学習目標
1. 微動観測の基本概念を理解する
2. SPAC法による位相速度の推定方法を習得する
3. 分散曲線から地下構造を推定する流れを理解する

## 必要なライブラリ
まず、必要なライブラリをインポートしましょう。